In [2]:
# Read data from h5 files and calculate moment arms
# Read all the h5 files in the folder
# Write the muscle list that have moment arm for each joint together with their strength and moment arms

import os
import h5py
import pandas as pd

# Specify the directory containing the .h5 files
directory_path = r'C:\Users\hs\Documents\AnyBody models\AMMR4-beta\Application\Validation\EvaluateMomentArms\Output'

def traverse_h5_group(group, path='', angular_velocity=None, side_prefix=''):
    """
    Traverse an HDF5 group recursively and collect data for each muscle.

    :param group: h5py.Group
        The HDF5 group to traverse.
    :param path: str
        The path used for recursive traversal.
    :param angular_velocity: float
        Angular velocity to be used for moment arm calculation.
    :param side_prefix: str
        Prefix indicating the side (e.g., "Right." or "Left.").
    :return: list
        A list of dictionaries containing muscle data.
    """
    data = []

    for key in group.keys():
        item = group[key]
        new_path = f"{path}.{key}" if path else key
        if isinstance(item, h5py.Group):
            data.extend(traverse_h5_group(item, new_path, angular_velocity, side_prefix))
        elif isinstance(item, h5py.Dataset):
            if key == 'Strength':
                # Process muscle name by removing 'Strength' and adding side prefix
                muscle_name = f"{side_prefix}{new_path.replace('.Strength', '')}"
                
                # Get the first value of the Strength array
                strength_array = item[()]
                if strength_array.size > 0:
                    strength = strength_array[0]  # Take only the first value
                else:
                    strength = None  # Handle cases where the array might be empty
                
                # Check if there's an 'LmtDot' dataset in the same group
                lmt_dot_array = None
                if 'LmtDot' in group:
                    lmt_dot_array = group['LmtDot'][()]  # Get the array directly

                # Initialize a dictionary to hold the muscle data
                entry = {'MuscleName': muscle_name, 'Strength': strength}
                
                # Add LmtDot values as separate columns if angular_velocity is available
                if lmt_dot_array is not None and angular_velocity is not None:
                    for i, lmt_dot in enumerate(lmt_dot_array):
                        if angular_velocity != 0:  # Prevent division by zero
                            entry[f'MomentArm_{i}'] = -lmt_dot / angular_velocity
                        else:
                            entry[f'MomentArm_{i}'] = None  # Handle division by zero case
                
                data.append(entry)

    return data

def get_angular_velocity(file_path):
    """
    Retrieve the angular velocity from the HDF5 file.

    :param file_path: str
        Path to the HDF5 file.
    :return: float or None
        Angular velocity if found, otherwise None.
    """
    try:
        with h5py.File(file_path, 'r') as h5_file:
            angular_velocity_path = 'Output/AngularVelocity_output/Val'
            if angular_velocity_path in h5_file:
                angular_velocity_dataset = h5_file[angular_velocity_path]
                if isinstance(angular_velocity_dataset, h5py.Dataset):
                    if angular_velocity_dataset.shape:
                        angular_velocity = angular_velocity_dataset[0]
                    else:
                        angular_velocity = angular_velocity_dataset[()]
                    return angular_velocity
                else:
                    print(f"'{angular_velocity_path}' is not a dataset.")
            else:
                print(f"Angular velocity path '{angular_velocity_path}' not found in {file_path}")

    except Exception as e:
        print(f"Error reading angular velocity from {file_path}: {e}")
    return None

def process_h5_file(file_path):
    """
    Process an HDF5 file and return a DataFrame with muscle data.

    :param file_path: str
        Path to the HDF5 file.
    :return: pd.DataFrame
        DataFrame containing the processed muscle data.
    """
    angular_velocity = get_angular_velocity(file_path)

    with h5py.File(file_path, 'r') as h5_file:
        muscle_groups = {
            'ShoulderArm': '',
            'Leg': '',
        }

        muscles_data = []
        
        # Iterate through the specified muscle groups
        for group_name, side_prefix in muscle_groups.items():
            try:
                base_group = h5_file['Output']['_Main']['HumanModel']['BodyModel']['Right'][group_name]['Mus']
                muscles_data.extend(traverse_h5_group(base_group, angular_velocity=angular_velocity, side_prefix=side_prefix))
            except KeyError:
                # If a group doesn't exist, skip it
                print(f"Group '{group_name}' not found in {file_path}, skipping.")
                continue

    # Create a DataFrame
    df = pd.DataFrame(muscles_data)

    # Remove columns where all values are None
    df.dropna(axis=1, how='all', inplace=True)

    # Filter out rows where any MomentArm_* column is less than 1e-6 or Strength is None
    moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
    for col in moment_arm_cols:
        df = df[df[col].apply(lambda x: x is not None and abs(x) >= 1e-6)]

    return df

def process_all_h5_files(directory):
    """
    Process all HDF5 files in a directory and write results to an Excel file.

    :param directory: str
        The directory to search for HDF5 files.
    """

    # Create a Pandas Excel writer using openpyxl as the engine
    with pd.ExcelWriter('muscle_MomentArm.xlsx', engine='openpyxl') as writer:
        data_written = False

        # Iterate over all files in the directory
        for filename in os.listdir(directory):
            if filename.endswith('.h5'):
                # Get the full path to the file
                file_path = os.path.join(directory, filename)
                
                # Process the file
                df = process_h5_file(file_path)
                
                if not df.empty:
                    sheet_name = os.path.splitext(filename)[0].split('_')[-1].replace('.anydata', '')
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    data_written = True
                    print(f"Processed {filename} and written to sheet '{sheet_name}'")
                else:
                    print(f"No valid data found in {filename}, skipping.")

        if not data_written:
            print("No data was written to the Excel file. Check the input files and directory.")

# Process all HDF5 files in the specified directory
process_all_h5_files(directory_path)

Group 'Leg' not found in C:\Users\hs\Documents\AnyBody models\AMMR4-beta\Application\Validation\EvaluateMomentArms\Output\MomentArms_Arm_Right_ACMedialRotation.anydata.h5, skipping.
Processed MomentArms_Arm_Right_ACMedialRotation.anydata.h5 and written to sheet 'ACMedialRotation'
Group 'Leg' not found in C:\Users\hs\Documents\AnyBody models\AMMR4-beta\Application\Validation\EvaluateMomentArms\Output\MomentArms_Arm_Right_ElbowFlexion.anydata.h5, skipping.
Processed MomentArms_Arm_Right_ElbowFlexion.anydata.h5 and written to sheet 'ElbowFlexion'
Group 'Leg' not found in C:\Users\hs\Documents\AnyBody models\AMMR4-beta\Application\Validation\EvaluateMomentArms\Output\MomentArms_Arm_Right_ElbowPronation.anydata.h5, skipping.
Processed MomentArms_Arm_Right_ElbowPronation.anydata.h5 and written to sheet 'ElbowPronation'
Group 'Leg' not found in C:\Users\hs\Documents\AnyBody models\AMMR4-beta\Application\Validation\EvaluateMomentArms\Output\MomentArms_Arm_Right_GHAbduction.anydata.h5, skipping

In [9]:
# ShoulderArm with mapping folder names: Create two anyscript files: list of all muscles spanning around the joints.

import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm_ShoulderArm.xlsx'

# Define the output file paths
output_file = 'List_Muscles_ShoulderArm_AMMR4.any'

# Dictionary to map joint names to folder names
joint_folder_map = {
    'ACMedialRotation': 'AcromioClavicularJoint',
    'ElbowFlexion': 'ElbowJoint',
    'ElbowPronation': 'PronationJoint',
    'GHFlexion': 'GlenohumeralJoint',
    'SCProtraction': 'SternoClavicularJoint',
    'WristFlexion': 'WristJoint',
    # Add more mappings if necessary
}

def write_header(file):
    """Write the conditional block header to the file."""
    file.write('AnyString shoulder_arm_muscle_folder = "Main.HumanModel.BodyModel." + Side + ".ShoulderArm.Mus.";\n\n')

def process_excel_file(excel_file, output_file):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Open file for writing
    with open(output_file, 'w') as file:
        # Write the header to the file
        write_header(file)

        for sheet_name in xls.sheet_names:
            # Check if the sheet_name is in the joint_folder_map
            if sheet_name in joint_folder_map:
                # Extract the joint name
                joint_name = sheet_name

                # Map the joint name to the folder name
                folder_name = joint_folder_map[joint_name]

                # Read the sheet into a DataFrame
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Ensure there is data in the DataFrame
                if not df.empty:
                    # Extract the muscle names (first column)
                    muscle_names = df.iloc[:, 0].tolist()

                    # Write joint muscles to the file
                    file.write(f'{folder_name} = {{\n')
                    if muscle_names:
                        # Split muscles into groups of 10
                        muscle_groups = [muscle_names[i:i+10] for i in range(0, len(muscle_names), 10)]
                        
                        arrcats = []
                        for group in muscle_groups:
                            muscles_str = ',\n'.join([f'    ObjSearch(.shoulder_arm_muscle_folder + "{muscle}")' for muscle in group])
                            arrcats.append(f'arrcat(\n{muscles_str}\n)')

                        if len(arrcats) > 10:
                            nested_arrcats = ['arrcat(\n' + ",\n".join(arrcats[i:i+10]) + '\n)' for i in range(0, len(arrcats), 10)]
                            final_arrcat = 'arrcat(\n' + ",\n".join(nested_arrcats) + '\n)'
                        else:
                            final_arrcat = 'arrcat(\n' + ",\n".join(arrcats) + '\n)'

                        file.write(f'  AnyObjectPtr List = {final_arrcat};\n')
                    else:
                        file.write('  AnyObjectPtr List = ObjSearch("NONE");\n')
                    file.write('};\n\n')

                    print(f"Processed joint {joint_name} from sheet {sheet_name}")
            else:
                print(f"Skipped sheet {sheet_name} as it is not in the joint_folder_map")

process_excel_file(excel_file, output_file)


Processed joint ACMedialRotation from sheet ACMedialRotation
Processed joint ElbowFlexion from sheet ElbowFlexion
Processed joint ElbowPronation from sheet ElbowPronation
Skipped sheet GHAbduction as it is not in the joint_folder_map
Skipped sheet GHExternalRotation as it is not in the joint_folder_map
Processed joint GHFlexion from sheet GHFlexion
Skipped sheet SCElevation as it is not in the joint_folder_map
Processed joint SCProtraction from sheet SCProtraction
Skipped sheet WristAbduction as it is not in the joint_folder_map
Processed joint WristFlexion from sheet WristFlexion


In [12]:
# Leg with mapping folder names: Create two anyscript files: list of all muscles spanning around the joints.

import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm_Leg.xlsx'

# Define the output file paths
output_file = 'List_Muscles_Leg_AMMR4.any'

# Dictionary to map joint names to folder names
joint_folder_map = {
    'AnklePlantarFlexion': 'AnkleJoint',
    'KneeFlexion': 'KneeJoint',
    'HipFlexion': 'HipJoint',
    'SubTalarEversion': 'SubTalarJoint',
    # Add more mappings if necessary
}

def write_header(file):
    """Write the conditional block header to the file."""
    file.write('AnyString leg_muscle_folder = "Main.HumanModel.BodyModel." + Side + ".Leg.Mus.";\n\n')

def process_excel_file(excel_file, output_file):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Open file for writing
    with open(output_file, 'w') as file:
        # Write the header to the file
        write_header(file)

        for sheet_name in xls.sheet_names:
            # Check if the sheet_name is in the joint_folder_map
            if sheet_name in joint_folder_map:
                # Extract the joint name
                joint_name = sheet_name

                # Map the joint name to the folder name
                folder_name = joint_folder_map[joint_name]

                # Read the sheet into a DataFrame
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Ensure there is data in the DataFrame
                if not df.empty:
                    # Extract the muscle names (first column)
                    muscle_names = df.iloc[:, 0].tolist()

                    # Write joint muscles to the file
                    file.write(f'{folder_name} = {{\n')
                    if muscle_names:
                        # Split muscles into groups of 10
                        muscle_groups = [muscle_names[i:i+10] for i in range(0, len(muscle_names), 10)]
                        
                        arrcats = []
                        for group in muscle_groups:
                            muscles_str = ',\n'.join([f'    ObjSearch(.leg_muscle_folder + "{muscle}")' for muscle in group])
                            arrcats.append(f'arrcat(\n{muscles_str}\n)')

                        if len(arrcats) > 10:
                            nested_arrcats = ['arrcat(\n' + ",\n".join(arrcats[i:i+10]) + '\n)' for i in range(0, len(arrcats), 10)]
                            final_arrcat = 'arrcat(\n' + ",\n".join(nested_arrcats) + '\n)'
                        else:
                            final_arrcat = 'arrcat(\n' + ",\n".join(arrcats) + '\n)'

                        file.write(f'  AnyObjectPtr List = {final_arrcat};\n')
                    else:
                        file.write('  AnyObjectPtr List = ObjSearch("NONE");\n')
                    file.write('};\n\n')

                    print(f"Processed joint {joint_name} from sheet {sheet_name}")
            else:
                print(f"Skipped sheet {sheet_name} as it is not in the joint_folder_map")

process_excel_file(excel_file, output_file)


Processed joint AnklePlantarFlexion from sheet AnklePlantarFlexion
Processed joint KneeFlexion from sheet KneeFlexion
Skipped sheet HipAbduction as it is not in the joint_folder_map
Skipped sheet HipExternalRotation as it is not in the joint_folder_map
Processed joint HipFlexion from sheet HipFlexion
Processed joint SubTalarEversion from sheet SubTalarEversion
